<a href="https://colab.research.google.com/github/DrSubbiah/python/blob/main/Upper_and_Lower_Bound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pathlib
import numpy as np
import glob
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Set path (drive)

path = r'/content/drive/MyDrive/XTS Data_Message Files/07112022'

#Fix the file pattern
filenames = glob.glob(path + "/*.csv")

#Read all files
dfs = [pd.read_csv(filename).assign(Time=os.path.basename(filename)) for filename in filenames]

#Concatenate all data into one DataFrame

df = pd.concat(dfs, ignore_index=False)

Making the time column readable

In [ ]:
df = df.reset_index(drop = True)
df['Time'] = df['Time'].str.rstrip('_message.csv')
df['Time'] = df['Time'].str.strip('_')
df = df.replace({'Time': {'_': ':'}}, regex=True)

check cycle condition

In [ ]:
df = df.loc[df['Cycle'] % 28 == 0]
df

,SYMBOL,Prev Signals,Signal,Signal.Condition,Trade Condition,Depth,Open,High,Low,Close,...,Low_sum,Close_sum,Qty,Min_Profit,Profit_flag,Sleep_Cycle,Sleep_Condition,Criteria,MESSAGE,Time
0,AUROPHARMA,SELL,Buy,False,True,18.2,550.9,550.6,550.1,550.4,...,550.1,550.4,187.9,534.8,True,28,True,NOT QUALIFIED,NaN,09:48:21
1,AXISBANK,BUY,Buy,False,True,28.1,870.8,871.0,870.3,870.3,...,870.3,870.3,118.7,846.4,True,28,True,NOT QUALIFIED,NaN,09:48:21
2,BERGEPAINT,SELL,Buy,False,True,20.5,605.2,605.6,605.0,605.5,...,605.0,605.5,170.9,588.0,True,28,True,NOT QUALIFIED,NaN,09:48:21
3,HAPPSTMNDS,SELL,Buy,False,True,32.6,976.8,976.8,975.7,976.7,...,975.7,976.7,105.9,948.8,True,28,True,NOT QUALIFIED,NaN,09:48:21
4,HDFCBANK,SELL,Buy,False,True,53.5,1500.0,1500.0,1499.6,1499.8,...,1499.6,1499.8,69.1,1453.4,True,28,True,NOT QUALIFIED,NaN,09:48:21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,SBIN,Sell,Sell,True,False,-1.8,609.0,609.0,608.7,609.0,...,608.7,609.0,163.7,607.7,True,140,True,NOT QUALIFIED,NaN,11:40:19
91,TATAMOTORS,Buy,Sell,False,False,-0.8,429.7,430.1,429.8,429.8,...,429.8,429.8,232.3,428.4,True,140,True,NOT QUALIFIED,NaN,11:40:19
92,TVSMOTOR,Buy,Sell,False,False,-5.9,1084.8,1084.8,1083.5,1083.7,...,1083.5,1083.7,91.8,1084.1,True,140,True,NOT QUALIFIED,NaN,11:40:19
93,WIPRO,Buy,Sell,False,False,-0.3,389.0,389.2,388.9,389.1,...,388.9,389.1,256.8,387.5,True,140,True,NOT QUALIFIED,NaN,11:40:19


creating and checking the conditions

In [ ]:
k = 1/6
dep = 0.9
df['b_ub'] = df['PP'] + (k*(df['PR'] - df['PP']))
df['s_lb'] = df['PP'] - (k*(df['PP'] - df['PS']))
df['b_lb'] = df['PP'] + dep
df['s_ub'] = df['PP'] - dep

In [ ]:
df['buy_lb_cond'] = np.where(df['Close'] >= df['b_lb'], "pass", "fail")
df['buy_ub_cond'] = np.where(df['Close'] <= df['b_ub'], "pass", "fail")
df['sell_ub_cond'] = np.where(df['Close'] <= df['s_ub'], "pass", "fail")
df['sell_lb_cond'] = np.where(df['Close'] >= df['s_lb'], "pass", "fail")

filtering out only the required columns

In [ ]:
BUYY = df[['SYMBOL', 'PS', 'PP', 'PR', 'Close', 'b_lb', 'b_ub', 's_lb', 's_ub', 'buy_lb_cond', 'buy_ub_cond', 'sell_lb_cond', 'sell_ub_cond', 'Depth', 'Time', 'MESSAGE']]
BUYY
df1 = BUYY

In [ ]:
BUYY

,SYMBOL,PS,PP,PR,Close,b_lb,b_ub,s_lb,s_ub,buy_lb_cond,buy_ub_cond,sell_lb_cond,sell_ub_cond,Depth,Time,MESSAGE
0,AUROPHARMA,526.4,532.1,537.9,550.4,533.0,533.066667,531.150000,531.2,pass,fail,pass,fail,18.2,09:48:21,NaN
1,AXISBANK,833.1,842.2,851.3,870.3,843.1,843.716667,840.683333,841.3,pass,fail,pass,fail,28.1,09:48:21,NaN
2,BERGEPAINT,578.7,585.1,591.4,605.5,586.0,586.150000,584.033333,584.2,pass,fail,pass,fail,20.5,09:48:21,NaN
3,HAPPSTMNDS,933.9,944.1,954.3,976.7,945.0,945.800000,942.400000,943.2,pass,fail,pass,fail,32.6,09:48:21,NaN
4,HDFCBANK,1439.0,1446.2,1453.4,1499.8,1447.1,1447.400000,1445.000000,1445.3,pass,fail,pass,fail,53.5,09:48:21,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,SBIN,604.2,610.8,617.4,609.0,611.7,611.900000,609.700000,609.9,fail,pass,fail,pass,-1.8,11:40:19,NaN
91,TATAMOTORS,425.9,430.6,435.2,429.8,431.5,431.366667,429.816667,429.7,fail,pass,fail,fail,-0.8,11:40:19,NaN
92,TVSMOTOR,1084.1,1089.6,1095.0,1083.7,1090.5,1090.500000,1088.683333,1088.7,fail,pass,fail,pass,-5.9,11:40:19,NaN
93,WIPRO,385.3,389.5,393.7,389.1,390.4,390.200000,388.800000,388.6,fail,pass,pass,fail,-0.3,11:40:19,NaN


pd.isnull to filter by NaN values

In [ ]:
df1 = df[~pd.isnull(df['MESSAGE'])]
call = list(df1.SYMBOL.drop_duplicates())

In [ ]:
call

['HDFCBANK',
 'HAPPSTMNDS',
 'INFY',
 'LUPIN',
 'BERGEPAINT',
 'ICICIBANK',
 'AXISBANK',
 'SBIN',
 'ADANIPORTS',
 'JSWSTEEL',
 'TATACONSUM']

In [ ]:
df5 = df.loc[(df['SYMBOL'] == 'ADANIPORTS')].index
df5

Int64Index([1, 22, 44, 65, 87, 109, 129, 149, 169, 189, 203, 222], dtype='int64')

Tried to get the indices and remove the rows which satisfied the above condition.

Did not work

In [ ]:
# for i in range(11):
#   ind = df.loc[(df['SYMBOL'] == str(call[i]))].index
  

In [ ]:
ind

Int64Index([16, 36, 56, 76, 96, 116, 136, 156, 176, 196, 205, 225], dtype='int64')

In [ ]:
df4 = df.loc[df['buy_lb_cond'] == 'fail']
df4 = df4[['SYMBOL', 'buy_lb_cond']]
df4

,SYMBOL,buy_lb_cond
1,ADANIPORTS,fail
2,AUROPHARMA,fail
3,AXISBANK,fail
4,BERGEPAINT,fail
5,HAPPSTMNDS,fail
...,...,...
233,HDFCLIFE,fail
235,KOTAKBANK,fail
237,TATAMOTORS,fail
238,TVSMOTOR,fail


In [ ]:
df2 = df.loc[df['SYMBOL'] == (call)]

ValueError: ignored

Writing the filtered df as csv to the path

In [ ]:
BUYY.to_csv(path + "/conditions.csv")